<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_12_Publish_Audio_Review_of_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest a PDF of an academic paper, use Sonnet 3.5 to write a comprehensive review of the paper, and listen to an audio rendition published using an [ElevenLabs](https://elevenlabs.io/) voice.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [ ]:
!pip install --upgrade graphlit-client

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [ ]:
from typing import List, Optional

# Create specification for Anthropic Sonnet 3.5
async def create_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Claude Sonnet 3.5",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        ),
        # NOTE: Optionally, ask LLM to revise it's response, which guarantees a full length and more detailed response
#        revisionStrategy=input_types.RevisionStrategyInput(
#            type=enums.RevisionStrategyTypes.CUSTOM,
#            customRevision="OK, that's not bad, but it needs more technical depth for this audience. You can do better than this. Reread all the context provided, and revise this into a longer, more thorough and compelling version. Don't mention anything about the revision.",
#            count=1
#        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def ingest_uri(uri: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def get_content(content_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.get_content(content_id)

        return response.content
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def publish_content(content_id: str, specification_id: str, prompt: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.publish_contents(
            name="Published Summary",
            connector=input_types.ContentPublishingConnectorInput(
               type=enums.ContentPublishingServiceTypes.ELEVEN_LABS_AUDIO,
               format=enums.ContentPublishingFormats.MP3,
               elevenLabs=input_types.ElevenLabsPublishingPropertiesInput(
                   model=enums.ElevenLabsModels.TURBO_V2_5,
                   voice="ZF6FPAbjXT4488VcRRnw" # ElevenLabs Amelia voice
               )
            ),
            summary_specification=input_types.EntityReferenceInput(
                id=specification_id
            ),
            publish_prompt = prompt,
            publish_specification=input_types.EntityReferenceInput(
                id=specification_id
            ),
            filter=input_types.ContentFilter(
                id=content_id
            ),
            is_synchronous=True
        )

        return response.publish_contents.content if response.publish_contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

In [ ]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

uri = "https://graphlitplatform.blob.core.windows.net/samples/Attention%20Is%20All%20You%20Need.1706.03762.pdf"
title = "Attention Is All You Need"
prompt = f"""
Speak as if you are a Ph.D. candidate who is reviewing a paper, and talking to your peers.

Follow these steps.

Step 1: Think about a structure for 10 minute long, engaging AI-generated paper review, with an welcome and introduction, an in-depth discussion of 4-6 interesting topics from the paper, and a wrap-up.
Step 2: For each topic, write 4-6 detailed paragraphs discussing it in-depth. Touch on key points for each topic which would be interesting to listeners. Mention the content metadata, entities and details from the provided summaries, as appropriate in the discussion. Remove any topic or section headings. Remove any references to podcast background music.  Remove any timestamps.
Step 3: Combine all topics into a lengthy, single-person script which can be used to record this audio review. Use friendly and compelling conversation to write the scripts.  You can be witty, but don't be cheesy.
Step 4: Remove any unnecessary formatting or final notes about being AI generated.

Refer to the content as the '{title}' paper.

Be specific when referencing persons, organizations, or any other named entities.
"""

specification_id = await create_specification()

if specification_id is not None:
    print(f'Created specification [{specification_id}]:')

    content_id = await ingest_uri(uri=uri)

    if content_id is not None:
        content = await get_content(content_id)

        if content is not None:
            display(Markdown(f'### Publishing Content [{content.id}]: {content.name}...'))

            published_content = await publish_content(content_id, specification_id, prompt)

            if published_content is not None:
                # Need to reload content to get presigned URL to MP3
                published_content = await get_content(published_content.id)

                if published_content is not None:
                    display(Markdown(f'### Published [{published_content.name}]({published_content.audio_uri})'))

                    display(HTML(f"""
                    <audio controls>
                    <source src="{published_content.audio_uri}" type="audio/mp3">
                    Your browser does not support the audio element.
                    </audio>
                    """))

                    display(Markdown('### Transcript'))
                    display(Markdown(published_content.markdown))
